In [ ]:
import os
import numpy as np
import scipy.io
import mne
from mne.time_frequency import tfr_multitaper
from sklearn.utils import resample
import matplotlib.pyplot as plt

In [ ]:
# Set directories
data_directory = 'E:\Research\EEG_Data\preprocessed_EEG_data\pre_aud-vis'
fieldTripDirectory = 'C:/Users/Mahmud/Documents/MATLAB/fieldtrip-20220729'

# Load files
files = [f for f in os.listdir(data_directory) if f.startswith('data_final') and f.endswith('.mat')]
nsubj = len(files)
data_list = []

def load_data(file_path):
    mat = scipy.io.loadmat(file_path)
    return mat['data_final']

In [ ]:
# Process each subject's data
for file in files:
    data_final = load_data(os.path.join(data_directory, file))
    
    # Extract different trial types based on 'trialinfo' values
    nona_trials = np.where(data_final['trialinfo'][:, 0] == 11)[0]
    unia_trials = np.where(data_final['trialinfo'][:, 0] == 21)[0]
    univ_trials = np.where(data_final['trialinfo'][:, 0] == 31)[0]
    nonv_trials = np.where(data_final['trialinfo'][:, 0] == 41)[0]
    
    # Create MNE Raw object from the data for each condition
    nona_raw = mne.io.RawArray(data_final['data'][:, nona_trials], mne.create_info(ch_names=list(data_final['label']), sfreq=1000))
    unia_raw = mne.io.RawArray(data_final['data'][:, unia_trials], mne.create_info(ch_names=list(data_final['label']), sfreq=1000))
    univ_raw = mne.io.RawArray(data_final['data'][:, univ_trials], mne.create_info(ch_names=list(data_final['label']), sfreq=1000))
    nonv_raw = mne.io.RawArray(data_final['data'][:, nonv_trials], mne.create_info(ch_names=list(data_final['label']), sfreq=1000))
    

In [ ]:
    # Frequency analysis using multitaper method
    freqs = np.arange(0.25, 8.25, 0.25)
    n_cycles = 3 / freqs
    
    nona_tfr = tfr_multitaper(nona_raw, freqs=freqs, n_cycles=n_cycles, time_bandwidth=2.0, return_itc=False)
    unia_tfr = tfr_multitaper(unia_raw, freqs=freqs, n_cycles=n_cycles, time_bandwidth=2.0, return_itc=False)
    univ_tfr = tfr_multitaper(univ_raw, freqs=freqs, n_cycles=n_cycles, time_bandwidth=2.0, return_itc=False)
    nonv_tfr = tfr_multitaper(nonv_raw, freqs=freqs, n_cycles=n_cycles, time_bandwidth=2.0, return_itc=False)
    
    # Baseline correction (decibel)
    nona_tfr.apply_baseline((-3, 0), mode='logratio')
    unia_tfr.apply_baseline((-3, 0), mode='logratio')
    univ_tfr.apply_baseline((-3, 0), mode='logratio')
    nonv_tfr.apply_baseline((-3, 0), mode='logratio')
    
    data_list.append((nona_tfr, unia_tfr, univ_tfr, nonv_tfr))

In [ ]:
# Compute Inter-Trial Phase Coherence (ITC)
def compute_itc(tfr):
    F = tfr.data  # shape (n_channels, n_freqs, n_times)
    N = F.shape[0]  # number of trials
    itc = np.abs(np.sum(F / np.abs(F), axis=0)) / N
    return itc ** 2 * N  # ITPCz

itc_list = []
for nona_tfr, unia_tfr, univ_tfr, nonv_tfr in data_list:
    nona_itc = compute_itc(nona_tfr)
    unia_itc = compute_itc(unia_tfr)
    univ_itc = compute_itc(univ_tfr)
    nonv_itc = compute_itc(nonv_tfr)
    itc_list.append((nona_itc, unia_itc, univ_itc, nonv_itc))

In [ ]:
# Grand average ITC
def grand_average(itc_data):
    return np.mean(np.array(itc_data), axis=0)

nona_itc_avg = grand_average([itc[0] for itc in itc_list])
unia_itc_avg = grand_average([itc[1] for itc in itc_list])
univ_itc_avg = grand_average([itc[2] for itc in itc_list])
nonv_itc_avg = grand_average([itc[3] for itc in itc_list])

In [ ]:
# Statistical analysis using permutation test
def permutation_test(data1, data2, n_permutations=2000, alpha=0.05):
    observed_diff = np.mean(data1) - np.mean(data2)
    permutation_diffs = []
    
    combined = np.concatenate((data1, data2))
    for _ in range(n_permutations):
        permuted = resample(combined, replace=False)
        perm_data1 = permuted[:len(data1)]
        perm_data2 = permuted[len(data1):]
        permutation_diffs.append(np.mean(perm_data1) - np.mean(perm_data2))
    
    p_value = np.sum(np.abs(permutation_diffs) >= np.abs(observed_diff)) / n_permutations
    return observed_diff, p_value

# Run statistical test
observed_diff, p_value = permutation_test(unia_itc_avg, nona_itc_avg)
print(f'Observed difference: {observed_diff:.4f}, p-value: {p_value:.4f}')

In [ ]:
# Visualize ITC
def plot_itc(itc_data, freqs, times, title):
    plt.figure(figsize=(10, 5))
    plt.imshow(itc_data, aspect='auto', extent=[times[0], times[-1], freqs[0], freqs[-1]], origin='lower', cmap='viridis')
    plt.colorbar(label='ITC')
    plt.title(title)
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.show()

# Plot ITC for each condition
plot_itc(nona_itc_avg, freqs, nona_tfr.times, 'ITC for Nona Trials')
plot_itc(unia_itc_avg, freqs, unia_tfr.times, 'ITC for Unia Trials')
plot_itc(univ_itc_avg, freqs, univ_tfr.times, 'ITC for Univ Trials')
plot_itc(nonv_itc_avg, freqs, nonv_tfr.times, 'ITC for Nonv Trials')